This file is to compile all (at least some) excels of 共同利用研究者連絡先（公募利用）and merge all contacts into one list.
# TODO list
- read H25,H26 and merge to list

In [13]:
import numpy as np
import pandas as pd
import os

In [14]:
# configurations
path = "../data/共同利用研究者連絡先（公募利用）"
skip = 2

In [15]:
file_list = os.listdir(path=path)
print(file_list)
# choose only 2 to -2
file_list = file_list[1:-1]
print(file_list)

['H26年度（大槌地区）.xlsx', 'H27 大槌・外来.xlsx', 'H28 大槌・外来.xlsx', 'H29 大槌・外来.xlsx', 'H30 大槌・外来.xlsx', 'R1大槌・外来.xlsx', 'R2 大槌・外来.xlsx', 'R3 大槌・外来.xlsx', 'R4 大槌・外来.xlsx', 'R5 大槌・外来.xlsx', 'Ｈ25年度（大槌地区）.xlsx']
['H27 大槌・外来.xlsx', 'H28 大槌・外来.xlsx', 'H29 大槌・外来.xlsx', 'H30 大槌・外来.xlsx', 'R1大槌・外来.xlsx', 'R2 大槌・外来.xlsx', 'R3 大槌・外来.xlsx', 'R4 大槌・外来.xlsx', 'R5 大槌・外来.xlsx']


In [16]:
class fileClass:
    def __init__(self,path,fn):
        self.path = path
        self.fn = fn
        return

    def readFile(self, skiprows):
        self.dt = pd.read_excel(f"{self.path}/{self.fn}", skiprows=skiprows)
        # remove FULL CHARACTER space in the column names
        self.dt.columns = self.dt.columns.str.replace("　", "")
        return self.dt

    def selectData(self, cols):
        self.dt_select = self.dt[cols]
        for i,col in enumerate(cols):
            ## remove \n and space in affliation
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace("　", "")
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace(" ", "")
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace("\n", "・")
        return self.dt_select


def loopAndStack(path, fn_list, skiprows, cols):
    # a function to stack all infomation (with redundence)
    fn0 = fn_list[0]
    file = fileClass(path=path,fn=fn0)
    file.readFile(skiprows=skiprows)
    list_all = file.selectData(cols=cols)
    for fn in fn_list[1:]:
        file = fileClass(path=path,fn=fn)
        file.readFile(skiprows=skiprows)
        list = file.selectData(cols=cols)
        # stack all
        list_all = np.vstack((list_all,list))
    return list_all

def removeRedundent(list_all):
    # a function to remove redundent names
    new_list = list_all[0:2,:]
    na = 0
    redun = 0
    # then loop every row to check if name exist
    for i,row in enumerate(list_all[1:,:]):
        if row[0] in new_list[:,0]:
            # if name already exist, pass
            redun += 1
            pass
        else:
            new_list = np.vstack((new_list,row))
    print(f"Org: {len(list_all)}; After: {len(new_list)}")
    print(f"Redundency: {redun}")
    return new_list


In [17]:
# Testing with one file
print("testing one case")
fn0 = file_list[0]
print(file_list)
print(fn0)
file = fileClass(path=path,fn=fn0)
file.readFile(skiprows=skip)
results = file.selectData(cols=['氏名','所属','職名', 'メールアドレス'])
#print(results)

testing one case
['H27 大槌・外来.xlsx', 'H28 大槌・外来.xlsx', 'H29 大槌・外来.xlsx', 'H30 大槌・外来.xlsx', 'R1大槌・外来.xlsx', 'R2 大槌・外来.xlsx', 'R3 大槌・外来.xlsx', 'R4 大槌・外来.xlsx', 'R5 大槌・外来.xlsx']
H27 大槌・外来.xlsx


c:\Users\Arthurli\anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [18]:
# read and save all: some names are repeated
colname = ['氏名','所属','職名', 'メールアドレス']
results = loopAndStack(path=path, fn_list=file_list, skiprows=skip, cols=colname)
print("Show full list (redundant): ")
contact_list_redun = pd.DataFrame(results, columns=colname)
contact_list_redun = contact_list_redun.dropna()
fn = 'contact_list_redun.xlsx'
contact_list_redun.to_excel(fn)
#########################################
#print(contact_list_redun)

Show full list (redundant): 


In [19]:
# to remove redundency
contact_list = removeRedundent(results)
print("Show new list (clean): ")
contact_list = pd.DataFrame(contact_list,columns=colname)
contact_list = contact_list.dropna()
######################################
#print(contact_list)

Org: 303; After: 88
Redundency: 216
Show new list (clean): 


In [20]:
# save the contact list
contact_list.to_excel('contact_list.xlsx')

Then, assign email address 

In [21]:
def checkAndAssign(df, contact_list,checking_by='氏名'):
    # a loop function that checks name and assign email
    for i,name in enumerate(df['PI-name']):
        if name in np.array(contact_list.iloc[:,0]):
            # assign email address: at col 7
            df.iloc[i,7] = str(np.array(contact_list[np.array(contact_list[checking_by])==name])[0][-1])
        else:
            #print(f"{name} not in list.")
            pass
    return df

In [22]:
# specify file to read, and check its column names
obs_fn = "ObservationLog_daily.xlsx"
df = pd.read_excel(obs_fn)
print(df.columns)

Index(['yyyy/mm/dd', 'Ship-name', 'PI-name', 'Pl-affiliation', 'PI-position',
       'Instruments', 'CTD (0/1)', 'PI-email', 'Unnamed: 8', 'Ship-name list'],
      dtype='object')


In [23]:
# assign email address to new df
new_df = checkAndAssign(df,contact_list)

In [24]:
# save as temporary excel file
new_df.to_excel('tmp.xlsx')

End of file